# Libraries

In [4]:
import ast
import gensim
import gensim.corpora as corpora 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
import scipy.cluster.hierarchy as shc

from bertopic import BERTopic
from collections import Counter
from gensim.models import CoherenceModel
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.neighbors._nearest_centroid import NearestCentroid

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data

In [5]:
file_path = "/Users/catherinepollack/Documents/dartmouth/research/aim3_facebook_covid19_obesity/data"
dat_obesity = "/220120_feature_matrix_no_bert.csv"

dat_obesity = pd.read_csv(str(file_path + dat_obesity))


/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (361,363,364,366,367,375,379,384,389,394,399,400,401,402) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Feature Matrix Curation

## Processing Metadata

In [3]:
loc = dat_obesity.columns.get_loc
dat_obesity_4 = dat_obesity.iloc[:,np.r_[loc("Likes at Posting"):loc("Followers at Posting")+1, 
                                         loc("Total Interactions"):loc("Care") + 1, 
                                         loc("Total Interactions (weighted  —  Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x )")]]
dat_obesity_4 = dat_obesity_4.replace(' ', '', regex=True)    
dat_obesity_4 = dat_obesity_4.fillna(0)                                    

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
loc = dat_obesity.columns.get_loc

dat_obesity_1 = dat_obesity.loc[:,"abnormal hormone":"years old"]
dat_obesity_2 = dat_obesity.loc[:,"neg":"compound"]
dat_obesity_3 = dat_obesity.loc[:,"WC":"OtherP"]
dat_obesity_fm = pd.concat([dat_obesity_1,
                            dat_obesity_2,
                            dat_obesity_3,
                            dat_obesity_4],
                            axis = 1)
print(dat_obesity_fm.shape) #522,473            


/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(522473, 463)


# Clustering

- Randomly Sample and Run Hierarchical Clustering, then run KMeans and DBSCAN
- LDA
- Pick Features Based on Reddit Analysis

## LDA Clustering

### Initializing Terms and Texts

In [5]:
texts = [ast.literal_eval(x) for x in dat_obesity.tokens]

terms = [item for sublist in texts for item in sublist]
terms = list(set(terms)) #106269

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Initial LDA Set-Up

In [6]:
# Create Dictionary
id2word = corpora.Dictionary(texts) 

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts] 

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Picking Best # of Topics

In [ ]:
# number of topics
for i in range(2, 26):
    lda_model_loop = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i,
                                           random_state = 110295)

    coherence_model_lda_loop = CoherenceModel(model=lda_model_loop, 
                                         texts=texts, 
                                         dictionary=id2word, 
                                         coherence='c_v')
    
    coherence_lda_loop = coherence_model_lda_loop.get_coherence()                                       
    print(f"Coherence of {coherence_lda_loop} for {i} topics.")

### Visualization

In [7]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14,
                                           random_state = 110295)

#vis = pyLDAvis.gensim_models.prepare(topic_model = lda_model, 
#                       corpus = corpus, 
#                       dictionary = id2word)
#pyLDAvis.enable_notebook()
#pyLDAvis.display(vis)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Export

In [39]:
#pyLDAvis.save_html(vis, 'lda.html')

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Exporting Clusters

In [10]:
topic_probabilities = []
[topic_probabilities.append(i) for i in lda_model[corpus]]

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [11]:
topic_probs_dict = {i: topic_probabilities[i] for i in range(0, len(topic_probabilities))}

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
test = pd.DataFrame.from_dict({k:dict(v) for k,v in topic_probs_dict.items()}, orient='index').reset_index()

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
#test.sort_values('index').to_csv(file_path + "220125_lda_topic_probabilities.csv")
test = test.sort_values('index')

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Bigrams

In [17]:
dat_obesity_test = dat_obesity.head()

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
def bigram(row):
    lst = ast.literal_eval(row['tokens'])
    return [(lst[x], lst[x+1].strip()) for x in range(len(lst)-1)]

dat_obesity["bigrams"] = dat_obesity.apply(lambda row: bigram(row), axis=1)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
dat_obesity["bigrams"].to_csv(file_path + "220127_facebook_post_bigrams.csv")

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
